In [3]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from nltk.lm import Vocabulary
import pickle
import sys
import torch
import importlib
from timeit import default_timer as t
sys.path.append("../../lib")
from metrics import levenshtein
import ocr_correction
from pytorch_decoding import seq2seq

In [4]:
folder = "../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish/BG/"

In [5]:
files = sorted(os.listdir(folder))
len(files)

1

In [6]:
import glob

files = glob.glob(folder + '/**/*.txt', recursive=True)
len(files)

49

In [7]:
data = []
for f in tqdm(files):
    with open(f) as file:
        data.append(file.readlines())

  0%|          | 0/49 [00:00<?, ?it/s]

In [8]:
data = pd.DataFrame(data, 
                    columns = ["ocr_to_input", 
                               "ocr_aligned", 
                               "gs_aligned"])\
.assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
        ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
        gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

print(data.shape)
data.head()

(49, 3)


,ocr_to_input,ocr_aligned,gs_aligned
0,"— 253 — тледахж, и азъ ги гледахъ. Прокуроръть...","@— 253 — тледахж, и азъ ги гледахъ. Прокурорът...","25 гледахѫ, и азъ ги гледахъ. Прокурорът..."
1,"3 1* противъ тоя новъ св&тъ, който не разбираш...","3 1* противъ тоя новъ св&тъ, който не разбираш...","@ @3 противъ тоя новъ свѣтъ, който не разбираш..."
2,ГОДИНА. КНИЖКА 4 АПРИЛ ИЙ 1890 ДЕННИ1ДА ИЗБОРЪ...,@ГОДИНА. @КНИЖКА 4 АПРИЛ ИЙ 1890 ДЕННИ1ДА ИЗБО...,ГОДИНА. АПРИЛИЙ @1890 КНИЖКА 4. ДЕННИ@ЦА ИЗБО...
3,Брой 6 и 7. ЖЕНСКИ ГЛАСЪ Стр. 15. двата отнесе...,@Брой 6 и 7. ЖЕНСКИ ГЛАСЪ Стр. 15. двата отнес...,Брой 6 и 7.@@@@@@@@@@@@@ Стр. 15@@@@@@@@@@@@@...
4,"374 МИКЕЛЪ АНДЖЕЛО Италиянски ваятель, архнтек...","374 МИКЕЛЪ АНДЖЕЛО Италиянски ваятель, архнтек...","@@@ МИКЕЛЪ АНДЖЕЛО Италиянски ваятель, архитек..."


In [9]:
data.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,49.000000,49.000000,49.000000
mean,3748.142857,3943.551020,3942.551020
std,2906.799056,3067.430983,3067.430983
min,803.000000,812.000000,811.000000
25%,1908.000000,2178.000000,2177.000000
50%,2922.000000,2935.000000,2934.000000
75%,4651.000000,4981.000000,4980.000000
max,13921.000000,14121.000000,14120.000000


In [10]:
vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))

193


In [11]:
distances = levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count    49.000000
mean     20.185595
std      21.140471
min       4.609723
25%       6.601283
50%       8.697515
75%      31.428571
max      79.192732
Name: cer, dtype: float64

In [12]:
distances = levenshtein(reference = data.gs_aligned, 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count    49.000000
mean     18.232816
std      18.112446
min       4.585742
25%       6.578947
50%       8.608991
75%      27.848101
max      72.935528
Name: cer, dtype: float64

In [14]:
data.to_pickle("../../data/bg/data/test.pkl")